In [3]:
import cv2
import jax
import tensorflow_datasets as tfds
import tqdm
import mediapy
import numpy as np
import os
os.environ['TOKENIZERS_PARALLELISM'] = 'false'
# from octo.utils.jax_utils import initialize_compilation_cache

# import tensorflow as tf
# initialize_compilation_cache()
# # prevent tensorflow from using GPU memory since it's only used for data loading
# tf.config.set_visible_devices([], "GPU")


In [1]:
import os
os.environ['HF_ENDPOINT'] = 'https://hf-mirror.com'
from huggingface_hub import hf_hub_download
from octo.model.octo_model import OctoModel
model = OctoModel.load_pretrained("hf://rail-berkeley/octo-small-1.5")
print('Successfully!')

/root/miniconda3/envs/octo/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-07-17 17:48:45.064784: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-17 17:48:45.064820: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-17 17:48:45.066341: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-17 17:48:45.961217: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Coul

Successfully!


In [4]:
# # create RLDS dataset builder
# builder = tfds.builder_from_directory(builder_dir='gs://gresearch/robotics/bridge/0.1.0/')
# ds = builder.as_dataset(split='train[:1]')
# 创建数据集构建器，使用本地路径
# builder = tfds.builder_from_directory(builder_dir='/root/octo/examples/bridge/0.1.0/')
builder = tfds.builder_from_directory(builder_dir='/root/autodl-tmp/aloha_sim_dataset/aloha_sim_cube_scripted_dataset/1.0.0/')
# builder = tfds.builder_from_directory(builder_dir='/root/autodl-tmp/tensorflow_datasets/duck_killer/1.0.0/')
# 加载数据集
ds = builder.as_dataset(split='train[:1]')
print(ds)

# sample episode + resize to 256x256 (default third-person cam resolution)
episode = next(iter(ds))
steps = list(episode['steps'])
images = [cv2.resize(np.array(step['observation']['top']), (256, 256)) for step in steps]
images = images[0:100]
print(len(images))
# extract goal image & language instruction
goal_image = images[-1]
language_instruction = steps[0]['language_instruction'].numpy().decode()

# visualize episode
print(f'Instruction: {language_instruction}')
# mediapy.show_video(images, fps=10)

<_PrefetchDataset element_spec={'episode_metadata': {'file_path': TensorSpec(shape=(), dtype=tf.string, name=None)}, 'steps': DatasetSpec({'action': TensorSpec(shape=(14,), dtype=tf.float32, name=None), 'discount': TensorSpec(shape=(), dtype=tf.float32, name=None), 'is_first': TensorSpec(shape=(), dtype=tf.bool, name=None), 'is_last': TensorSpec(shape=(), dtype=tf.bool, name=None), 'is_terminal': TensorSpec(shape=(), dtype=tf.bool, name=None), 'language_instruction': TensorSpec(shape=(), dtype=tf.string, name=None), 'observation': {'state': TensorSpec(shape=(14,), dtype=tf.float32, name=None), 'top': TensorSpec(shape=(480, 640, 3), dtype=tf.uint8, name=None)}, 'reward': TensorSpec(shape=(), dtype=tf.float32, name=None)}, TensorShape([]))}>
100
Instruction: pick up the cube and hand it over


In [5]:
WINDOW_SIZE = 2

# create `task` dict
task = model.create_tasks(goals={"image_primary": goal_image[None]})   # for goal-conditioned
task = model.create_tasks(texts=[language_instruction])                  # for language conditioned

In [6]:
# run inference loop, this model only uses 3rd person image observations for bridge
# collect predicted and true actions
pred_actions, true_actions = [], []
for step in tqdm.trange(len(images) - (WINDOW_SIZE - 1)):
    input_images = np.stack(images[step:step+WINDOW_SIZE])[None]
    observation = {
        'image_primary': input_images,
        'timestep_pad_mask': np.full((1, input_images.shape[1]), True, dtype=bool)
    }
    
    # this returns *normalized* actions --> we need to unnormalize using the dataset statistics
    actions = model.sample_actions(
        observation, 
        task, 
        unnormalization_statistics=model.dataset_statistics["bridge_dataset"]["action"], 
        rng=jax.random.PRNGKey(0)
    )
    actions = actions[0] # remove batch dim

    pred_actions.append(actions)
    final_window_step = step + WINDOW_SIZE - 1
    true_actions.append(np.concatenate(
        (
            steps[final_window_step]['action']['world_vector'], 
            steps[final_window_step]['action']['rotation_delta'], 
            np.array(steps[final_window_step]['action']['open_gripper']).astype(np.float32)[None]
        ), axis=-1
    ))

  0%|          | 0/99 [00:00<?, ?it/s]WARNING:root:'observations' is missing items compared to example_batch: {'pad_mask_dict/timestep', 'timestep', 'pad_mask_dict/image_primary', 'image_wrist', 'pad_mask_dict/image_wrist', 'task_completed'}
2024-07-17 17:49:42.010935: W external/xla/xla/service/gpu/conv_algorithm_picker.cc:810] None of the algorithms provided by cuDNN heuristics worked; trying fallback algorithms.
2024-07-17 17:49:42.010955: W external/xla/xla/service/gpu/conv_algorithm_picker.cc:813] Conv: (f32[2,32,128,128]{3,2,1,0}, u8[0]{0}) custom-call(f32[2,6,256,256]{3,2,1,0}, f32[32,6,3,3]{3,2,1,0}, f32[32]{0}), window={size=3x3 stride=2x2 pad=1_1x1_1}, dim_labels=bf01_oi01->bf01, custom_call_target="__cudnn$convBiasActivationForward", backend_config={"conv_result_scale":1,"activation_mode":"kNone","side_input_scale":0,"leakyrelu_alpha":0}
  0%|          | 0/99 [00:11<?, ?it/s]


XlaRuntimeError: UNKNOWN: Failed to determine best cudnn convolution algorithm for:
%cudnn-conv-bias-activation.15 = (f32[2,32,128,128]{3,2,1,0}, u8[0]{0}) custom-call(f32[2,6,256,256]{3,2,1,0} %bitcast.3392, f32[32,6,3,3]{3,2,1,0} %divide.268, f32[32]{0} %Arg_232.233), window={size=3x3 stride=2x2 pad=1_1x1_1}, dim_labels=bf01_oi01->bf01, custom_call_target="__cudnn$convBiasActivationForward", metadata={op_name="jit(sample_actions)/jit(main)/jit(run_transformer)/octo_transformer/observation_tokenizers_primary/SmallStem16_0/StdConv_0/conv_general_dilated[window_strides=(2, 2) padding=((1, 1), (1, 1)) lhs_dilation=(1, 1) rhs_dilation=(1, 1) dimension_numbers=ConvDimensionNumbers(lhs_spec=(0, 3, 1, 2), rhs_spec=(3, 2, 0, 1), out_spec=(0, 3, 1, 2)) feature_group_count=1 batch_group_count=1 precision=None preferred_element_type=None]" source_file="/root/octo/octo/model/components/vit_encoders.py" source_line=126}, backend_config={"conv_result_scale":1,"activation_mode":"kNone","side_input_s